## folium
[https://folium.readthedocs.org/en/latest/](https://folium.readthedocs.org/en/latest/)

In [ ]:
%pylab inline

import pandas as pd
import numpy as np
import folium
from IPython.display import HTML

def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [ ]:
df = pd.read_csv('starbucks.csv')

In [ ]:
tileset = 'http://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png'
#tileset = 'http://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}.png'
#tileset="http://{s}.tile.stamen.com/watercolor/{z}/{x}/{y}.jpg"
map_osm = folium.Map(location=[37.5491,126.98955], tiles=tileset, zoom_start=13, attr='cartocdn')
#map_osm = folium.Map(location=[37.5491,126.98955], zoom_start=13)

for idx, p in df.iterrows():
    p.coord = p.coord.replace('[', '')
    p.coord = p.coord.replace(']', '')
    lat, lng = p.coord.split(',')
    map_osm.circle_marker(location=(lat, lng), radius=50,
                  popup='My Popup Info', line_color='#d92015',
                  fill_color='#d92015', fill_opacity=1)
    
embed_map(map_osm)

In [ ]:
def marker_map(df, tileset=None, zoom=13, cluster=False):
    if not tileset:
        tileset = 'http://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png'
    
    map_osm = folium.Map(location=[37.5491,126.98955], tiles=tileset, zoom_start=zoom, attr='cartocdn')

    for idx, p in df.iterrows():
        p.coord = p.coord.replace('[', '')
        p.coord = p.coord.replace(']', '')
        lat, lng = p.coord.split(',')
        map_osm.simple_marker(location=(lat, lng), popup=p['name'], clustered_marker=cluster)
    return map_osm


s = pd.read_csv('starbucks.csv')
e = pd.read_csv('ediya.csv')
c = pd.read_csv('caffebene.csv')
a = pd.read_csv('angelinus.csv')
df = pd.concat([s, e, c, a])

tileset = "http://{s}.tile.stamen.com/watercolor/{z}/{x}/{y}.jpg"
# -0-  cluster 인자(True/False)를 사용해봅니다.
embed_map(marker_map(df, tileset=tileset, zoom=9, cluster=False))

In [ ]:
import pyproj
WGS84 = pyproj.Proj(proj='latlong', datum='WGS84', ellps='WGS84')
TM128 = pyproj.Proj(proj='tmerc', lat_0='38N', lon_0='128E', ellps='bessel', x_0='400000', y_0='600000', k='0.9999', towgs84='-146.43,507.89,681.46')
GRS80 = pyproj.Proj(proj='tmerc', lat_0='38N', lon_0='127.5E', k='0.9996', x_0='1000000', y_0='2000000', ellps='GRS80', units='m')
TM127 = pyproj.Proj(proj='tmerc', lat_0='38N', lon_0='127.0028902777777777776E', ellps='bessel', x_0='200000', y_0='500000', k='1.0', towgs84='-146.43,507.89,681.46')

def wgs84_to_tm128(longitude, latitude):
    lon, lat = pyproj.transform(WGS84, TM128, longitude, latitude )
    return lat, lon

def wgs84_to_tm127(longitude, latitude):
    lon, lat = map(lambda x:2.5*x,
        pyproj.transform(WGS84, TM127, longitude, latitude ))
    return lat, lon

def tm127_to_wgs84(x, y):
    lon, lat = pyproj.transform(TM127, WGS84, x/2.5, y/2.5 )
    return lat, lon

# UTM-K
def grs80_to_wgs84(x, y):
    lon, lat = pyproj.transform(GRS80, WGS84, x, y )
    return lat, lon

# Naver (카텍)
def tm128_to_wgs84(x, y):
    lon, lat = pyproj.transform(TM128, WGS84, x, y)
    return lat, lon


## geopandas
[http://geopandas.org/](http://geopandas.org/)

### 설치
기본적인 가이드는 아래 링크를 참조합니다.
http://geoffboeing.com/2014/09/using-geopandas-windows/

위 가이드에서는 다음과 같은 순서로 설치할 것을 권장합니다.
먼저 아래 링크에서 pyproj, GDAL, Fiona, Shapely 패키지를 찾아서 다운 받습니다.
http://www.lfd.uci.edu/~gohlke/pythonlibs/

패키지 파일 이름은 패키지명-cp파이썬버전-os_bit.whl
ex) pyproj‑1.9.4‑cp34‑none‑win_amd64.whl
파이썬 버전은 IPython이나 IPython notebook에서 (왜냐하면 IPython에서 사용할거니까요)

```
import sys
sys.version
```
을 실행해서 확인합니다.

위 4가지 패키지 (.whl 파일)를 적절한 곳에 다운로드 받은 다음 cmd (명령행 프롬프트)를 이용해서 전체 파일이름을 주어 설치합니다.

```
C:\> dir 다운받은경로
pip install pyproj‑1.9.4‑cp34‑none‑win_amd64.whl
```
같은 방식으로 GDAL, Fiona, Shapely를 순서대로 설치합니다.

모두 설치가 성공적으로 끝나면 pip install geopandas를 실행해서 geopandas를 설치합니다.

제대로 설치되었는지 확인은:
```
import geopandas as gp
gp.version.version
```
을 실행해서 설치된 버전이 제대로 출력되면 성공입니다.


http://www.gisdeveloper.co.kr/entry/%EC%B5%9C%EC%8B%A0%EC%8B%A0-%ED%96%89%EC%A0%95%EA%B5%AC%EC%97%AD%EB%8F%84-%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C

In [ ]:
import geopandas as gp
gdf = gp.GeoDataFrame.from_file('kor_shp.shp', encoding='euc-kr')

In [ ]:
gdf.head()

In [ ]:
gn = gdf[gdf['SIG_KOR_NM'] == '강남구'].geometry
#type(gn.iloc[0])
#list(gn.iloc[0].exterior.coords)
gn_polygons = [grs80_to_wgs84(x, y) for x, y in gn.iloc[0].exterior.coords]

In [ ]:
gn.iloc[0].exterior

In [ ]:
gw = gdf[gdf['SIG_KOR_NM'] == '관악구'].geometry
gw_polygons = [grs80_to_wgs84(x, y) for x, y in gw.iloc[0].exterior.coords]
tileset = 'http://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png'

map_osm = folium.Map(location=[37.5491,126.98955], tiles=tileset, zoom_start=11, attr='cartocdn')
map_osm.line(locations=gn_polygons)
map_osm.line(locations=gw_polygons)

embed_map(map_osm)